In [2]:
# Import libraries
import pandas as pd
from mlxtend.frequent_patterns import apriori ,association_rules
import matplotlib.pyplot as plt
import seaborn as snsno

C:\Users\s.rasti\Anaconda3\lib\site-packages\seaborn\rcmod.py:82: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(mpl.__version__) >= "3.0":
C:\Users\s.rasti\Anaconda3\lib\site-packages\setuptools\_distutils\version.py:346: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)
C:\Users\s.rasti\Anaconda3\lib\site-packages\seaborn\rcmod.py:82: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(mpl.__version__) >= "3.0":
C:\Users\s.rasti\Anaconda3\lib\site-packages\setuptools\_distutils\version.py:346: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)


In [3]:
#  Load data from an Excel file named "Basket_Cosmetic.xlsx"
df_sale_product=pd.read_excel("Basket_Cosmetic.xlsx")

In [4]:
#Check the shape (number of rows and columns) of the dataset
df_sale_product.shape

(11957, 10)

(11957, 10)

In [5]:
# Get information about the dataset, including data types and non-null counts
df_sale_product.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11957 entries, 0 to 11956
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   OrderId                11957 non-null  int64  
 1   QtyFinal               11957 non-null  int64  
 2   SaleAmount             11957 non-null  float64
 3   CountOrder             11957 non-null  int64  
 4   AttributeSetName       11957 non-null  object 
 5   AttributeSetId         11957 non-null  int64  
 6   CategoryName_Level2    11925 non-null  object 
 7   ProductMiddleCategory  11957 non-null  object 
 8   ProductName            11957 non-null  object 
 9   ProductCode            11957 non-null  int64  
dtypes: float64(1), int64(5), object(4)
memory usage: 934.3+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11957 entries, 0 to 11956
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------

In [6]:
# ProductRel222=ProductRel['SaleAmount']

In [7]:
# display the first two rows and the last row of the data to get an overview.
df_sale_product.head(2)
df_sale_product.tail(2)

,OrderId,QtyFinal,SaleAmount,CountOrder,AttributeSetName,AttributeSetId,CategoryName_Level2,ProductMiddleCategory,ProductName,ProductCode
11955,358677,1,102000.0,1,اکسسوري آرايشي,64,لوازم آرایش,نامشخص,شانه دندانه درشت اوريفليم Oriflame مدل Wide To...,613048
11956,372394,1,520000.0,1,مراقبت پوست,34,مراقبت و بهداشت پوست,نامشخص,سرم آبرسان صورت کپسولي نوتروژينا Neutrogena سر...,635777


,OrderId,QtyFinal,SaleAmount,CountOrder,AttributeSetName,AttributeSetId,CategoryName_Level2,ProductMiddleCategory,ProductName,ProductCode
11955,358677,1,102000.0,1,اکسسوري آرايشي,64,لوازم آرایش,نامشخص,شانه دندانه درشت اوريفليم Oriflame مدل Wide To...,613048
11956,372394,1,520000.0,1,مراقبت پوست,34,مراقبت و بهداشت پوست,نامشخص,سرم آبرسان صورت کپسولي نوتروژينا Neutrogena سر...,635777


In [8]:
# Generate descriptive statistics for the dataset, including count, mean, min, max, etc.
df_sale_product.describe(include = 'all')

,OrderId,QtyFinal,SaleAmount,CountOrder,AttributeSetName,AttributeSetId,CategoryName_Level2,ProductMiddleCategory,ProductName,ProductCode
count,11957.000000,11957.000000,1.195700e+04,11957.000000,11957,11957.000000,11925,11957,11957,11957.000000
unique,NaN,NaN,NaN,NaN,15,NaN,12,4,4542,NaN
top,NaN,NaN,NaN,NaN,آرايشي و بهداشتي,NaN,لوازم آرایش,آرايشي و بهداشتي,ريمل حجم دهنده اسنس مدل I Love Extreme Volume,NaN
freq,NaN,NaN,NaN,NaN,6257,NaN,4109,6009,72,NaN
mean,367995.044075,1.149703,1.584498e+05,1.000418,NaN,27.511081,NaN,NaN,NaN,529612.477712
std,8681.272803,0.900728,2.100535e+05,0.020446,NaN,13.257505,NaN,NaN,NaN,114428.737443
min,352672.000000,0.000000,0.000000e+00,1.000000,NaN,17.000000,NaN,NaN,NaN,198221.000000
25%,360444.000000,1.000000,4.900000e+04,1.000000,NaN,17.000000,NaN,NaN,NaN,483952.000000
50%,368109.000000,1.000000,9.800000e+04,1.000000,NaN,17.000000,NaN,NaN,NaN,542768.000000
75%,375584.000000,1.000000,1.950000e+05,1.000000,NaN,34.000000,NaN,NaN,NaN,613850.000000


,OrderId,QtyFinal,SaleAmount,CountOrder,AttributeSetName,AttributeSetId,CategoryName_Level2,ProductMiddleCategory,ProductName,ProductCode
count,11957.000000,11957.000000,1.195700e+04,11957.000000,11957,11957.000000,11925,11957,11957,11957.000000
unique,NaN,NaN,NaN,NaN,15,NaN,12,4,4542,NaN
top,NaN,NaN,NaN,NaN,آرايشي و بهداشتي,NaN,لوازم آرایش,آرايشي و بهداشتي,ريمل حجم دهنده اسنس مدل I Love Extreme Volume,NaN
freq,NaN,NaN,NaN,NaN,6257,NaN,4109,6009,72,NaN
mean,367995.044075,1.149703,1.584498e+05,1.000418,NaN,27.511081,NaN,NaN,NaN,529612.477712
std,8681.272803,0.900728,2.100535e+05,0.020446,NaN,13.257505,NaN,NaN,NaN,114428.737443
min,352672.000000,0.000000,0.000000e+00,1.000000,NaN,17.000000,NaN,NaN,NaN,198221.000000
25%,360444.000000,1.000000,4.900000e+04,1.000000,NaN,17.000000,NaN,NaN,NaN,483952.000000
50%,368109.000000,1.000000,9.800000e+04,1.000000,NaN,17.000000,NaN,NaN,NaN,542768.000000
75%,375584.000000,1.000000,1.950000e+05,1.000000,NaN,34.000000,NaN,NaN,NaN,613850.000000


In [9]:
# Insert a new column named 'quantity' with a default value of 1 at position 10
df_sale_product.insert(10, 'quantity',1)

In [11]:
# Group and pivot the data to create a basket of products by OrderId
product_basket = (df_sale_product.groupby(['OrderId', 'ProductCode'])['quantity']).sum().unstack().reset_index().fillna(0).set_index('OrderId')

In [12]:
# Define a function to convert values to 0 if they are less than or equal to 0, and 1 if greater than or equal to 1
def convto0(x):
    if (x<=0):
        return 0
    if (x>=1):
        return 1
# Apply the conversion function to the product_basket dataset
Basket_sets = product_basket.applymap(convto0)
Basket_sets.head()

ProductCode,198221,198223,198232,198233,198254,198256,198258,201143,217506,249729,...,731167,732019,732254,732255,732256,732257,732258,732259,732260,732261
OrderId,,,,,,,,,,,,,,,,,,,,,
352672,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
352685,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
352697,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
352740,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
352743,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


ProductCode,198221,198223,198232,198233,198254,198256,198258,201143,217506,249729,...,731167,732019,732254,732255,732256,732257,732258,732259,732260,732261
OrderId,,,,,,,,,,,,,,,,,,,,,
352672,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
352685,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
352697,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
352740,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
352743,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
# Use the Apriori algorithm to mine frequent itemsets from the basket dataset
frequent_itemsets = apriori(Basket_sets.astype('bool'), min_support = 0.0008, use_colnames = True)
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets

,support,itemsets,length
0,0.001006,(198232),1
1,0.001761,(249729),1
2,0.004779,(249796),1
3,0.001258,(250527),1
4,0.001509,(250584),1
...,...,...,...
960,0.001006,"(439906, 439909, 439917)",3
961,0.001006,"(439913, 439906, 439917)",3
962,0.001006,"(439913, 439909, 439917)",3
963,0.001006,"(439913, 439917, 439910)",3


,support,itemsets,length
0,0.001006,(198232),1
1,0.001761,(249729),1
2,0.004779,(249796),1
3,0.001258,(250527),1
4,0.001509,(250584),1
...,...,...,...
960,0.001006,"(439906, 439909, 439917)",3
961,0.001006,"(439913, 439906, 439917)",3
962,0.001006,"(439913, 439909, 439917)",3
963,0.001006,"(439913, 439917, 439910)",3


In [14]:
# Filter the itemsets where the length is greater than or equal to 1
lenght=frequent_itemsets['itemsets'].str.len()
Filter1=lenght>=1
frequent_itemsets[Filter1]

,support,itemsets,length
0,0.001006,(198232),1
1,0.001761,(249729),1
2,0.004779,(249796),1
3,0.001258,(250527),1
4,0.001509,(250584),1
...,...,...,...
960,0.001006,"(439906, 439909, 439917)",3
961,0.001006,"(439913, 439906, 439917)",3
962,0.001006,"(439913, 439909, 439917)",3
963,0.001006,"(439913, 439917, 439910)",3


,support,itemsets,length
0,0.001006,(198232),1
1,0.001761,(249729),1
2,0.004779,(249796),1
3,0.001258,(250527),1
4,0.001509,(250584),1
...,...,...,...
960,0.001006,"(439906, 439909, 439917)",3
961,0.001006,"(439913, 439906, 439917)",3
962,0.001006,"(439913, 439909, 439917)",3
963,0.001006,"(439913, 439917, 439910)",3


In [17]:
# Generate association rules from the frequent itemsets based on support
association_rules_data = association_rules(frequent_itemsets, metric = 'support', min_threshold = 0.0008)
association_rules_data

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(250971),(612964),0.005030,0.006539,0.001006,0.200000,30.584615,0.000973,1.241826
1,(612964),(250971),0.006539,0.005030,0.001006,0.153846,30.584615,0.000973,1.175873
2,(253867),(255655),0.008048,0.005785,0.001006,0.125000,21.608696,0.000959,1.136246
3,(255655),(253867),0.005785,0.008048,0.001006,0.173913,21.608696,0.000959,1.200784
4,(259042),(253867),0.005785,0.008048,0.001006,0.173913,21.608696,0.000959,1.200784
...,...,...,...,...,...,...,...,...,...
177,"(584242, 584254)",(584235),0.001006,0.004276,0.001006,1.000000,233.882353,0.001002,inf
178,"(584235, 584254)",(584242),0.001006,0.003773,0.001006,1.000000,265.066667,0.001002,inf
179,(584242),"(584235, 584254)",0.003773,0.001006,0.001006,0.266667,265.066667,0.001002,1.362264
180,(584235),"(584242, 584254)",0.004276,0.001006,0.001006,0.235294,233.882353,0.001002,1.306377


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(250971),(612964),0.005030,0.006539,0.001006,0.200000,30.584615,0.000973,1.241826
1,(612964),(250971),0.006539,0.005030,0.001006,0.153846,30.584615,0.000973,1.175873
2,(253867),(255655),0.008048,0.005785,0.001006,0.125000,21.608696,0.000959,1.136246
3,(255655),(253867),0.005785,0.008048,0.001006,0.173913,21.608696,0.000959,1.200784
4,(259042),(253867),0.005785,0.008048,0.001006,0.173913,21.608696,0.000959,1.200784
...,...,...,...,...,...,...,...,...,...
177,"(584242, 584254)",(584235),0.001006,0.004276,0.001006,1.000000,233.882353,0.001002,inf
178,"(584235, 584254)",(584242),0.001006,0.003773,0.001006,1.000000,265.066667,0.001002,inf
179,(584242),"(584235, 584254)",0.003773,0.001006,0.001006,0.266667,265.066667,0.001002,1.362264
180,(584235),"(584242, 584254)",0.004276,0.001006,0.001006,0.235294,233.882353,0.001002,1.306377


In [18]:
association_rules_data.dtypes

antecedents            object
consequents            object
antecedent support    float64
consequent support    float64
support               float64
confidence            float64
lift                  float64
leverage              float64
conviction            float64
dtype: object

antecedents            object
consequents            object
antecedent support    float64
consequent support    float64
support               float64
confidence            float64
lift                  float64
leverage              float64
conviction            float64
dtype: object

In [ ]:
# Export the rules to an Excel file named 'BasketAnalysisOutput.xlsx'
excell=association_rules_data.to_excel('BasketAnalysisOutput.xlsx')